In [3]:
#from elasticsearch import Elasticsearch
from pyelasticsearch import ElasticSearch #see if this is better than 'official API' - the docs look better to me
import json
from jsmin import jsmin

In [5]:
%install_ext https://raw.githubusercontent.com/szeitlin/watermark/master/watermark.py

Installed watermark.py. To use it, type:
  %load_ext watermark


In [6]:
%load_ext watermark

In [7]:
%watermark -a "Samantha Zeitlin" -d -u -p elasticsearch,pyelasticsearch,jsmin,python

Samantha Zeitlin 
Last updated: 04/09/2015 

elasticsearch 1.4.0
pyelasticsearch 1.2.1
jsmin 2.1.1
python 2.7.9


In [8]:
#this could be made more general later, should decide how many nbs to put per index(?), whether to specify mapping, id numbers or let ES do that
prefix = '{"index": {"_index": "test", "_type": "type1"}}\n'

In [15]:
%%bash
rm pyminified_testfile.json

In [16]:
#try doing two files together (if this works, put target files in folder and iterate through that)
with open("elk_test/import-one-year.ipynb", 'r') as test:
    minified = jsmin(test.read())
with open("elk_test/2013_2014_comparison_prep.ipynb", 'r') as test2:
    mini2 = jsmin(test2.read())
    with open('pyminified_testfile.json', 'a+') as pyminified:
        pyminified.write(prefix)
        pyminified.write(minified)
        pyminified.write(prefix)
        pyminified.write(mini2)

In [17]:
#official elasticsearch API wants an iterator
def mini_iter(action_source_pairs_file):
    for line in action_source_pairs_file:
        yield line

In [18]:
count=0
with open('pyminified_testfile.json', 'r') as pyminified:
    for line in mini_iter(pyminified):
        count+=1
print(count)

3


In [59]:
# with open('pyminified_testfile.json', 'r') as pyminified:
#     new = mini_iter(pyminified)
#     next(new)

In [11]:
es = ElasticSearch('http://localhost:9200/')
#es.indices.create(index='test-index')

In [64]:
#es.indices.delete(index='test-index')

{u'acknowledged': True}

In [48]:
#print(elasticsearch.Elasticsearch.bulk.__doc__)


        Perform many index/delete operations in a single API call.
        `<http://www.elasticsearch.org/guide/en/elasticsearch/reference/current/docs-bulk.html>`_

        See the :func:`~elasticsearch.helpers.bulk` helper function for a more
        friendly API.

        :arg body: The operation definition and data (action-data pairs), as
            either a newline separated string, or a sequence of dicts to
            serialize (one per row).
        :arg index: Default index for items which don't provide one
        :arg doc_type: Default document type for items which don't provide one
        :arg consistency: Explicit write consistency setting for the operation
        :arg refresh: Refresh the index after performing the operation
        :arg routing: Specific routing value
        :arg replication: Explicitly set the replication type (default: sync)
        :arg timeout: Explicit operation timeout
        


In [26]:
print es.bulk.__doc__


        Perform multiple index, delete, create, or update actions per request.

        Used with helper routines :meth:`index_op()`, :meth:`delete_op()`, and
        :meth:`update_op()`, this provides an efficient, readable way to do
        large-scale changes. This contrived example illustrates the structure::

            es.bulk([es.index_op({'title': 'All About Cats', 'pages': 20}),
                     es.index_op({'title': 'And Rats', 'pages': 47}),
                     es.index_op({'title': 'And Bats', 'pages': 23})],
                    doc_type='book',
                    index='library')

        More often, you'll want to index (or delete or update) a larger number
        of documents. In those cases, yield your documents from a generator,
        and use :func:`~pyelasticsearch.bulk_chunks()` to divide them into
        multiple requests::

            from pyelasticsearch import bulk_chunks

            def documents():
                for book in books:
              

In [25]:
print es.index_op.__doc__


        Return a document-indexing operation that can be passed to
        :meth:`bulk()`. (See there for examples.)

        Specifically, return a 2-line, JSON-encoded bytestring.

        :arg doc: A mapping of property names to values.
        :arg doc_type: The type of the document to index, if different from
            the one you pass to :meth:`bulk()`
        :arg overwrite_existing: Whether we should overwrite existing
            documents of the same ID and doc type. (If False, this does a
            `create` operation.)
        :arg meta: Other args controlling how the document is indexed,
            like ``id`` (most common), ``index`` (next most common),
            ``version``, and ``routing``. See `ES's bulk API`_ for details on
            these.

        .. _`ES's bulk API`:
            http://www.elastic.co/guide/en/elasticsearch/reference/current/docs-bulk.html

        


In [29]:
# with open('pyminified_testfile.json', 'r') as pyminified:
#     es.bulk((es.index_op(line for line in pyminified), index='test', doc_type='notebook')

In [66]:
!ls

Dockerfile                                 another_query.txt                          first_try_mapping.json                     minified_testfile.json                     testcreate.json
LICENSE                                    draft_bulk_import_multiple_notebooks.ipynb import_statements_query.txt                pyminified_testfile.json                   testnb.json
README.md                                  elasticsearch.yml                          match_query.txt                            test2.json                                 testprefix.json
action.json                                elk_test                                   minified.json                              testbulk.json                              testquery.txt


In [28]:
# with open('import_statements_query.txt', 'r') as query:
#     q=query.read()
#     es.search(q, index='test')

In [21]:
%%bash
curl -s -XPOST localhost:9200/_bulk --data-binary @test2.json


{"took":251,"errors":false,"items":[{"index":{"_index":"test","_type":"type1","_id":"2","_version":1,"status":201}}]}

In [ ]:
#looks like kibana barely works at all with only 1 document, suddenly with 2 it starts showing more functionality. 